In [134]:
"""
Determining Propagation Geometry and Turbulence Conditions
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

In [135]:
#Determine geometry
D2 = 0.5 #diameter of the observation aperture [m]
wvl = 1e-6 #optical wavelength [m]
k = 2*np.pi / wvl #optical wavenumber [rad/m]
Dz = 50e3 #propagation distance [m]
R = Dz #wavefront radius of curvature [m]

In [136]:
#Use Sinc to model pt source
DROI = 4 * D2
D1 = wvl*Dz / DROI 
R = Dz 

In [137]:
#Atmospheric properties
Cn2 = 1e-16 #structure parameter [m^-2/3], constant

In [138]:
#SW and PW coherence diameters [m]
r0sw = (0.423 * k**2 * Cn2 * 3.0/8 * Dz)**(-3.0/5)
r0pw = (0.423 * k**2 * Cn2 * Dz)**(-3.0/5)
p = np.linspace(0, Dz, 1000)

In [139]:
#log-amplitude variance
rytov = 0.563 * k**(7.0/6) * np.sum( Cn2 * (1 - p/Dz)**(5.0/6) * p**(5.0/6) * (p[1] - p[0]) )

In [140]:
#screen properties
nscr = 11 #number of screens

A = np.zeros((2,nscr))

alpha = np.arange(0,nscr)/(nscr-1)
print(alpha)
A[0] = alpha**(5.0/3)
print(A[0])
A[1] = (1 - alpha)**(5.0/6) * alpha**(5.0/6)
print(A[1])
B = np.asarray([r0sw**(-5.0/3), rytov/1.33 * (k/Dz)**(5.0/6)])

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
[0.         0.02154435 0.06839904 0.13444214 0.21715341 0.31498026
 0.4268272  0.55186146 0.6894191  0.83895278 1.        ]
[0.         0.13444214 0.21715341 0.27238472 0.30444537 0.31498026
 0.30444537 0.27238472 0.21715341 0.13444214 0.        ]


In [141]:
#initial guess
x0 = 0.001*(nscr/3*r0sw * np.ones(nscr))**(-5.0/6)

In [142]:
#constraints:
#1) x>0 (absolutely neccessary - implemented)
#2) rmax = 0.1 (maximum Rytov number per partial group - not absolutely necessary? - not implemented)

In [143]:
#objective function

def fun(x, A, B):
    return B - A.dot(x)

#minimise using scipy.optimize.least_squares - allows contraint that x>0 through setting bounds
res = least_squares(fun, x0, args =(A,B), bounds = (0,np.inf))

In [144]:
x = res.x
print(x, B - A.dot(x), r0sw, r0pw, rytov)

[1.89506668e-03 3.13032749e+00 5.32060127e+00 7.05775270e+00
 8.41672651e+00 9.42161535e+00 1.00739124e+01 1.03573149e+01
 1.02302683e+01 9.58783672e+00 2.34445945e-02] [ 1.15052146e-09 -4.43010251e-09] 0.12664241226501954 0.07030689768053816 0.4364896658907703


In [145]:
"""
Sampling Constraints

Obtained from exercise_sampling_constraints_angular_spectrum_partial_propagation_method_with_turbulence.ipynb
"""

delta1 = 0.01
deltan = 0.01

N = 512